In [1]:
"""
This script reads two CSV files containing data previously scraped with the "Data scraper Inderes" code, merges them into a single DataFrame, and preprocesses the data for analysis.

1. Reading CSV Files:
   - The script handles errors while reading the CSV files by first attempting to read the entire file. 
     If an error occurs, it falls back to reading only the first 1000 rows to inspect the data.

2. Merging DataFrames:
   - After successfully reading both datasets, the script merges them into a single DataFrame using the pandas 'concat' function. 
     It then renames the 'Likes' column to 'Post Likes' for clarity.

3. Copying DataFrame:
   - To ensure data integrity, the script creates a copy of the merged DataFrame for further processing.

4. Preprocessing Data:
   - The script converts string representations of dictionaries in the 'Comments Details' column to actual dictionaries using the 'ast' module.
   - It explodes the 'Comments Details' column to create a new row for each comment, and extracts specific details from the dictionaries.
   - Comments are aggregated while merging with a unique DataFrame.

5. Handling Date and Timestamps:
   - Finnish dates and timestamps are converted into standard formats (with four-digit years) using predefined mappings.
   - Time-related columns are converted to datetime objects for ease of analysis.

6. Identifying Company Mentions:
   - The script identifies company mentions in comments using pre-defined patterns and Named Entity Recognition (NER) classification.
   - It processes comments to extract validated matches of company mentions.

7. Converting Numeric Representations:
   - Mixed numeric representations are converted to integers for consistency and ease of analysis.

8. Saving Preprocessed Data:
   - The preprocessed DataFrame is saved to a pickle file for future use.

9. Execution Time:
   - The script calculates the elapsed time for execution and prints it for monitoring purposes.

Note:
- Ensure that all necessary libraries are installed before running the script.
- Adjust the file paths and other parameters as needed.
"""

import time
import re
import pandas as pd
import ast
from datetime import datetime, timedelta
from transformers import pipeline

# Define the file paths for the datasets
file_path1 = 'my_datadf5.csv'
file_path2 = 'my_datadf6.csv'

# Start time to measure execution duration
start_time = time.time()

def read_csv_with_error_handling(file_path):
    """
    Read a CSV file with error handling.

    Parameters:
        file_path (str): The file path of the CSV file to be read.

    Returns:
        pd.DataFrame or None: A DataFrame containing the data read from the CSV file,
        or None if an error occurs during reading.

    Raises:
        FileNotFoundError: If the specified file does not exist.
        IOError: If an I/O error occurs while reading the file.
    """
    try:
        df = pd.read_csv(file_path, error_bad_lines=False)
    except Exception as e:
        print(f"Error reading the full file {file_path}:", e)
        # If full read fails, try reading only the first 1000 rows to inspect the data
        try:
            df = pd.read_csv(file_path, nrows=1000)
        except Exception as e:
            print(f"Error reading a subset of file {file_path}:", e)
            df = None
    return df

# Read the datasets with error handling
data_df5 = read_csv_with_error_handling(file_path1)
data_df6 = read_csv_with_error_handling(file_path2)

# Check if both dataframes are loaded successfully
if data_df5 is not None and data_df6 is not None:
    # Display the first few rows of each dataset to inspect the data
    print("First few rows of DataFrame 5:")
    print(data_df5.head())
    print("\nFirst few rows of DataFrame 6:")
    print(data_df6.head())

    # Concatenate the dataframes
    merged_df = pd.concat([data_df5, data_df6], ignore_index=True)
    print("\nMerged DataFrame:")
    print(merged_df.head())
else:
    print("One or both files could not be loaded properly.")


<ipython-input-1-1bb46326dc0d>:31: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, error_bad_lines=False)
<ipython-input-1-1bb46326dc0d>:31: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, error_bad_lines=False)


First few rows of DataFrame 5:
                                                 URL Created At Last Reply  \
0  https://keskustelut.inderes.fi/t/clear-blue-te...   joulu 20      23 pv   
1  https://keskustelut.inderes.fi/t/synnex-corpor...   heinä 21   heinä 21   
2  https://keskustelut.inderes.fi/t/spekulaatio-o...    kesä 20   joulu 23   
3  https://keskustelut.inderes.fi/t/vesiosakkeet/...    kesä 19       1 pv   
4   https://keskustelut.inderes.fi/t/nintendo-co-ltd  marras 20       2 pv   

     Visits Replies  Title Users  Likes  Links  \
0   40283.0     229    NaN    53  1,3 k   50.0   
1    1513.0       4    NaN     4      5    2.0   
2  236849.0   1,4 k    NaN   230  2,8 k   50.0   
3   10685.0      26    NaN    12     55   12.0   
4   36924.0     268    NaN    73  1,2 k   50.0   

                                    Comments Details  
0  [{'post_id': 'post_1', 'comment': 'Avataanpa k...  
1  [{'post_id': 'post_1', 'comment': 'En sopivaa ...  
2  [{'post_id': 'post_1', 'comment

In [2]:
# Rename the 'Likes' column to 'Post Likes'
merged_df.rename(columns={'Likes': 'Post Likes'}, inplace=True)
small_df = merged_df.copy()

In [3]:
small_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860 entries, 0 to 1859
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   URL               1860 non-null   object 
 1   Created At        1824 non-null   object 
 2   Last Reply        1824 non-null   object 
 3   Visits            1801 non-null   float64
 4   Replies           1824 non-null   object 
 5   Title             0 non-null      float64
 6   Users             1815 non-null   object 
 7   Post Likes        1823 non-null   object 
 8   Links             1739 non-null   float64
 9   Comments Details  1860 non-null   object 
dtypes: float64(3), object(7)
memory usage: 145.4+ KB


In [4]:

# Function to safely evaluate the literal or return an empty dictionary in case of error
def safe_literal_eval(x):
    """
    Safely evaluate a literal expression from a string representation.

    Parameters:
        x (str): The string representation to evaluate.

    Returns:
        dict: A dictionary containing the evaluated literal expression,
        or an empty dictionary if the evaluation fails.
    """
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return {}

# Define a custom function to extract details or return None for missing values
def extract_details(x, key):
    """
    Extract details from a dictionary or return None if the key is missing.

    Parameters:
        x (dict): The dictionary containing the details.
        key (str): The key to extract from the dictionary.

    Returns:
        Any: The value corresponding to the key if it exists, otherwise None.
    """
    if isinstance(x, dict):
        return x.get(key)
    else:
        return None

# Convert the string representations in 'Comments Details' column to dictionaries
small_df['Comments Details'] = small_df['Comments Details'].apply(safe_literal_eval)

# Explode the 'Comments Details' column to create a new row for each comment
expanded_df = small_df.explode('Comments Details')

# Use apply to directly extract details or return None for missing values
expanded_df['Comment'] = expanded_df['Comments Details'].apply(lambda x: extract_details(x, 'comment'))
expanded_df['Timestamp'] = expanded_df['Comments Details'].apply(lambda x: extract_details(x, 'timestamp'))
expanded_df['Likes'] = expanded_df['Comments Details'].apply(lambda x: extract_details(x, 'likes'))
expanded_df['post_id'] = expanded_df['Comments Details'].apply(lambda x: extract_details(x, 'post_id'))

# Define a function to aggregate comments and merge with unique dataframe
def aggregate_comments(df):
    """
    Aggregate comments and merge with a unique dataframe.

    Parameters:
        df (pd.DataFrame): The DataFrame containing comments.

    Returns:
        pd.DataFrame: The DataFrame with aggregated comments merged with a unique dataframe.
    """
    # Create a subset for aggregation
    comment_subset = df[['URL', 'post_id', 'Comment']]

    # Aggregate comments
    aggregated_comments = comment_subset.groupby(['URL', 'post_id'])['Comment'].agg(' '.join).reset_index()

    # Remove duplicates from the original dataframe
    unique_df = df.drop_duplicates(subset=['URL', 'post_id']).drop('Comment', axis=1)

    # Merge the unique dataframe with the aggregated comments
    optimized_df = unique_df.merge(aggregated_comments, on=['URL', 'post_id'], how='inner')

    # Create Comment ID
    optimized_df['Comment ID'] = optimized_df['URL'] + '_' + optimized_df['post_id']

    return optimized_df.drop(['URL', 'post_id'], axis=1)

# Call the function to aggregate comments and merge with unique dataframe
optimized_df = aggregate_comments(expanded_df)

# Reset index
optimized_df.reset_index(drop=True, inplace=True)

# Drop the "Comments Details" column
optimized_df.drop(columns=['Comments Details'], inplace=True)

# Display the first few rows
print(optimized_df.head())


  Created At Last Reply   Visits Replies  Title Users Post Likes  Links  \
0   joulu 20      23 pv  40283.0     229    NaN    53      1,3 k   50.0   
1   joulu 20      23 pv  40283.0     229    NaN    53      1,3 k   50.0   
2   joulu 20      23 pv  40283.0     229    NaN    53      1,3 k   50.0   
3   joulu 20      23 pv  40283.0     229    NaN    53      1,3 k   50.0   
4   joulu 20      23 pv  40283.0     229    NaN    53      1,3 k   50.0   

                   Timestamp  Likes  \
0  13. joulukuuta 2020 18.15   32.0   
1  13. joulukuuta 2020 18.22   15.0   
2  13. joulukuuta 2020 21.37    4.0   
3  13. joulukuuta 2020 21.56    4.0   
4   14. joulukuuta 2020 9.46   16.0   

                                             Comment  \
0  Avataanpa ketju Clear Blue Technologies Intern...   
1  Kiitokset erinomaisesta aloituksesta! Ketjua C...   
2  Iso kiitos täältäkin ketjun aloittajalle! Viim...   
3  Hyvä avaus. Onko kukaan vielä laskenut tunnusl...   
4  Kiitokset! Yritystä ei seuraa v

In [5]:
optimized_df = optimized_df.drop('Title', axis=1)

optimized_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256795 entries, 0 to 256794
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Created At  256777 non-null  object 
 1   Last Reply  256777 non-null  object 
 2   Visits      256731 non-null  float64
 3   Replies     256777 non-null  object 
 4   Users       254216 non-null  object 
 5   Post Likes  256774 non-null  object 
 6   Links       256184 non-null  float64
 7   Timestamp   256795 non-null  object 
 8   Likes       256795 non-null  float64
 9   Comment     256795 non-null  object 
 10  Comment ID  256795 non-null  object 
dtypes: float64(3), object(8)
memory usage: 21.6+ MB


In [6]:

# Step 1: Replace nulls in 'Links' with 0
optimized_df['Links'] = optimized_df['Links'].fillna(0)

# Step 2: Drop rows where any of the other columns is null
# Exclude 'Links' from the columns checked for nulls
columns_to_check = optimized_df.columns.difference(['Links'])
optimized_df = optimized_df.dropna(subset=columns_to_check)


In [7]:
optimized_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254167 entries, 0 to 256794
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Created At  254167 non-null  object 
 1   Last Reply  254167 non-null  object 
 2   Visits      254167 non-null  float64
 3   Replies     254167 non-null  object 
 4   Users       254167 non-null  object 
 5   Post Likes  254167 non-null  object 
 6   Links       254167 non-null  float64
 7   Timestamp   254167 non-null  object 
 8   Likes       254167 non-null  float64
 9   Comment     254167 non-null  object 
 10  Comment ID  254167 non-null  object 
dtypes: float64(3), object(8)
memory usage: 23.3+ MB


In [8]:
df = optimized_df.copy()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254167 entries, 0 to 256794
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Created At  254167 non-null  object 
 1   Last Reply  254167 non-null  object 
 2   Visits      254167 non-null  float64
 3   Replies     254167 non-null  object 
 4   Users       254167 non-null  object 
 5   Post Likes  254167 non-null  object 
 6   Links       254167 non-null  float64
 7   Timestamp   254167 non-null  object 
 8   Likes       254167 non-null  float64
 9   Comment     254167 non-null  object 
 10  Comment ID  254167 non-null  object 
dtypes: float64(3), object(8)
memory usage: 23.3+ MB


In [10]:
# Define regular expression patterns for company mentions
all_company_patterns = {
    "Neste Oyj": r"(?i)\bneste\w*",
    "KONE Oyj": r"(?i)\bkone\w*",
    "Sampo Oyj": r"(?i)\bsampo\w*",
    "UPM-Kymmene Oyj": r"(?i)\bupm\w*",
    "Nokia Corporation": r"(?i)\bnokia\w*",
    "Fortum Oyj": r"(?i)\bfortum\w*",
    "Stora Enso Oyj": r"(?i)\bstora\w*enso\w*",
    "Wärtsilä Oyj Abp": r"(?i)\bwärtsilä\w*",
    "Metso Outotec Oyj": r"(?i)\bmetso\w*",
    "Kesko Oyj": r"(?i)\bkesko\w*",
    "Elisa Oyj": r"(?i)\belisa\w*",
    "Orion Oyj": r"(?i)\borion\w*",
    "Valmet Oyj": r"(?i)\bvalmet\w*",
    "Huhtamäki Oyj": r"(?i)\bhuhtamäki\w*",
    "Cargotec Corp": r"(?i)\bcargotec\w*",
    "Konecranes Plc": r"(?i)\bkonecranes\w*",
    "Kojamo Oyj": r"(?i)\bkojamo\w*",
    "Metsä Board Oyj": r"(?i)\bmetsä\w*",
    "Kemira Oyj": r"(?i)\bkemira\w*",
    "TietoEVRY Oyj": r"(?i)\btieto\w*",
    "Uponor Oyj": r"(?i)\buponor\w*",
    "Outokumpu Oyj": r"(?i)\boutokumpu\w*",
    "QT Group Oyj": r"(?i)\bqt\w*",
    "Fiskars OYJ Abp": r"(?i)\bfiskars\w*",
    "Vaisala Oyj": r"(?i)\bvaisala\w*",
    "Caverion Oyj": r"(?i)\bcaverion\w*",
    "Nokian Renkaat Oyj": r"(?i)\bnokian\w*",
    "Sanoma Oyj": r"(?i)\bsanoma\w*",
    "Enento Group Oyj": r"(?i)\benento\w*",
    "Terveystalo Oyj": r"(?i)\bterveystalo\w*",
    "Citycon Oyj": r"(?i)\bcitycon\w*",
    "Tokmanni Group Oyj": r"(?i)\btokmanni\w*",
    "Musti Group Oyj": r"(?i)\bmusti\w*",
    "Alma Media Oyj": r"(?i)\balma\w*",
    "Finnair Oyj": r"(?i)\bfinnair\w*",
    "Oma Säästöpankki Oyj": r"(?i)\boma\w*säästöpankki\w*",
    "Revenio Group Oyj": r"(?i)\brevenio\w*",
    "Ponsse Oyj": r"(?i)\bponsse\w*",
    "eQ Oyj": r"(?i)\beq\w*",
    "Olvi Oyj": r"(?i)\bolvi\w*",
    "Evli Pankki Oyj": r"(?i)\bevli\w*",
    "Marimekko Oyj": r"(?i)\bmarimekko\w*",
    "Harvia Oyj": r"(?i)\bharvia\w*",
    "Scanfil Oyj": r"(?i)\bscanfil\w*",
    "Ålandsbanken Abp": r"(?i)\bålandsbanken\w*",
    "Lassila & Tikanoja Oyj": r"(?i)\blassila\w*",
    "CapMan Oyj": r"(?i)\bcapman\w*",
    "Etteplan Oyj": r"(?i)\betteplan\w*",
    "Gofore Oyj": r"(?i)\bgofore\w*",
    "Viking Line ABP": r"(?i)\bviking\w*line\w*",
    "Raisio plc": r"(?i)\braisio\w*",
    "Talenom Oyj": r"(?i)\btalenom\w*",
    "Atria Oyj": r"(?i)\batria\w*",
    "Relais Group Oyj": r"(?i)\brelais\w*",
    "Incap Oyj": r"(?i)\bincap\w*",
    "Admicom Oyj": r"(?i)\badmicom\w*",
    "Kamux Oyj": r"(?i)\bkamux\w*",
    "Detection Technology Oyj": r"(?i)\bdetection\w*technology\w*",
    "Oriola Oyj": r"(?i)\boriola\w*",
    "Aspo Oyj": r"(?i)\baspo\w*",
    "NoHo Partners Oyj": r"(?i)\bnoho\w*",
    "Titanium Oyj": r"(?i)\btitanium\w*",
    "Bittium Oyj": r"(?i)\bbittium\w*",
    "Suominen Oyj": r"(?i)\bsuominen\w*",
    "Pihlajalinna Oyj": r"(?i)\bpihlajalinna\w*",
    "Verkkokauppa.com Oyj": r"(?i)\bverkkokauppa\w*",
    "Sitowise Group Oyj": r"(?i)\bsitowise\w*",
    "Afarak Group Oyj": r"(?i)\bafarak\w*",
    "Tecnotree Oyj": r"(?i)\btecnotree\w*",
    "Keskisuomalainen Oyj": r"(?i)\bkeskisuomalainen\w*",
    "Nurminen Logistics Oyj": r"(?i)\bnurminen\w*",
    "Consti Oyj": r"(?i)\bconsti\w*"
}

# Initialize the NER (Named Entity Recognition) model
model_checkpoint = "Kansallisarkisto/finbert-ner"
ner_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="simple")

# Pre-compile regular expressions for efficient pattern matching
compiled_patterns = {company_name: re.compile(pattern, re.IGNORECASE) for company_name, pattern in all_company_patterns.items()}

def process_comments(comment, compiled_patterns, ner_classifier):
    """
    Process comments to extract mentions of companies using pre-defined patterns and NER classification.

    Parameters:
        comment (str): The comment to process.
        compiled_patterns (dict): A dictionary containing pre-compiled regular expression patterns for company mentions.
        ner_classifier: The NER classifier model used to identify organization names.

    Returns:
        list: A list of tuples containing validated matches of company mentions in the comment.
              Each tuple contains the matched text, company name, and a boolean indicating if it's an organization.
    """
    validated_matches = []
    
    # Loop through precompiled patterns and search for matches
    for company_name, pattern in compiled_patterns.items():
        match = pattern.search(comment)
        if match:
            match_text = match.group()
            
            # Use NER classifier to check if the match is an organization name
            ner_results = ner_classifier(match_text)
            is_org = any(entity_info['entity_group'] == 'ORG' for entity_info in ner_results)
            
            # Append the validated match to the list
            if is_org:
                validated_matches.append((match_text, company_name, is_org))
    
    # Print the validated matches after processing the entire comment
    if validated_matches:
        print("Processed comment:", validated_matches)
    
    return validated_matches

# Apply processing to each comment in the DataFrame
print("Processing comments in DataFrame...")
df['Matched Companies Info'] = df['Comment'].apply(lambda x: process_comments(x, compiled_patterns, ner_classifier))
print("Processing complete.")

Processing comments in DataFrame...
Processed comment: [('Elisan', 'Elisa Oyj', True)]
Processed comment: [('Reveniosta', 'Revenio Group Oyj', True), ('Talenom', 'Talenom Oyj', True)]
Processed comment: [('Sampoon', 'Sampo Oyj', True)]
Processed comment: [('Valmet', 'Valmet Oyj', True)]
Processed comment: [('Kemira', 'Kemira Oyj', True)]
Processed comment: [('Tokmannilta', 'Tokmanni Group Oyj', True)]
Processed comment: [('Fortum', 'Fortum Oyj', True)]
Processed comment: [('UPM', 'UPM-Kymmene Oyj', True)]
Processed comment: [('Metson', 'Metso Outotec Oyj', True)]
Processed comment: [('Metson', 'Metso Outotec Oyj', True)]
Processed comment: [('Metson', 'Metso Outotec Oyj', True)]
Processed comment: [('Metson', 'Metso Outotec Oyj', True)]
Processed comment: [('Metson', 'Metso Outotec Oyj', True)]
Processed comment: [('Metson', 'Metso Outotec Oyj', True)]
Processed comment: [('Metsoa', 'Metso Outotec Oyj', True)]
Processed comment: [('Metsolla', 'Metso Outotec Oyj', True)]
Processed comme

In [11]:
data = df.copy()
data.head()

,Created At,Last Reply,Visits,Replies,Users,Post Likes,Links,Timestamp,Likes,Comment,Comment ID,Matched Companies Info
0,joulu 20,23 pv,40283.0,229,53,"1,3 k",50.0,13. joulukuuta 2020 18.15,32.0,Avataanpa ketju Clear Blue Technologies Intern...,https://keskustelut.inderes.fi/t/clear-blue-te...,[]
1,joulu 20,23 pv,40283.0,229,53,"1,3 k",50.0,13. joulukuuta 2020 18.22,15.0,Kiitokset erinomaisesta aloituksesta! Ketjua C...,https://keskustelut.inderes.fi/t/clear-blue-te...,[]
2,joulu 20,23 pv,40283.0,229,53,"1,3 k",50.0,13. joulukuuta 2020 21.37,4.0,Iso kiitos täältäkin ketjun aloittajalle! Viim...,https://keskustelut.inderes.fi/t/clear-blue-te...,[]
3,joulu 20,23 pv,40283.0,229,53,"1,3 k",50.0,13. joulukuuta 2020 21.56,4.0,Hyvä avaus. Onko kukaan vielä laskenut tunnusl...,https://keskustelut.inderes.fi/t/clear-blue-te...,[]
4,joulu 20,23 pv,40283.0,229,53,"1,3 k",50.0,14. joulukuuta 2020 9.46,16.0,Kiitokset! Yritystä ei seuraa vielä yksikään a...,https://keskustelut.inderes.fi/t/clear-blue-te...,[]


In [12]:
# Filter out rows with empty "Matched Companies Info"
data = data[data['Matched Companies Info'].apply(lambda x: len(x) > 0)]

# Reset the index after filtering
data.reset_index(drop=True, inplace=True)

data.head()

,Created At,Last Reply,Visits,Replies,Users,Post Likes,Links,Timestamp,Likes,Comment,Comment ID,Matched Companies Info
0,joulu 20,23 pv,40283.0,229,53,"1,3 k",50.0,23. joulukuuta 2020 21.29,4.0,Tämä on kyllä ihan eri use case kuin mobiilive...,https://keskustelut.inderes.fi/t/clear-blue-te...,"[(Elisan, Elisa Oyj, True)]"
1,kesä 20,joulu 23,236849.0,"1,4 k",230,"2,8 k",50.0,11. kesäkuuta 2020 7.46,5.0,Toki monella on juuri tuota pikavoittosysteemi...,https://keskustelut.inderes.fi/t/spekulaatio-o...,"[(Reveniosta, Revenio Group Oyj, True), (Talen..."
2,kesä 20,joulu 23,236849.0,"1,4 k",230,"2,8 k",50.0,11. kesäkuuta 2020 11.22,5.0,tai toimintaan osallistuvista riippumattomiin ...,https://keskustelut.inderes.fi/t/spekulaatio-o...,"[(Sampoon, Sampo Oyj, True)]"
3,kesä 20,joulu 23,236849.0,"1,4 k",230,"2,8 k",50.0,13. heinäkuuta 2020 12.09,0.0,Itse en ollut edes Fiskeristä kuullut ennen tä...,https://keskustelut.inderes.fi/t/spekulaatio-o...,"[(Valmet, Valmet Oyj, True)]"
4,kesä 19,1 pv,10685.0,26,12,55,12.0,29. lokakuuta 2023 10.47,1.0,Kemira ketjussa jo kyselin PFAS kemikaaleista....,https://keskustelut.inderes.fi/t/vesiosakkeet/...,"[(Kemira, Kemira Oyj, True)]"


In [13]:


# Adjust the current_date if your context is not today
specific_datetime = datetime(2024, 1, 28, 12, 33, 56, 971723)

# Mapping for Finnish month abbreviations
month_mapping = {
    'tammi': '01', 'helmi': '02', 'maalis': '03', 'huhti': '04', 'touko': '05',
    'kesä': '06', 'heinä': '07', 'elo': '08', 'syys': '09', 'loka': '10',
    'marras': '11', 'joulu': '12',
    'tammikuuta': '01', 'helmikuuta': '02', 'maaliskuuta': '03', 'huhtikuuta': '04',
    'toukokuuta': '05', 'kesäkuuta': '06', 'heinäkuuta': '07', 'elokuuta': '08',
    'syyskuuta': '09', 'lokakuuta': '10', 'marraskuuta': '11', 'joulukuuta': '12'
}

# Function to convert Finnish date to standard format (with four-digit year)
def convert_date(date_str):
    # Handle absolute dates
    for fin_month, num_month in month_mapping.items():
        if fin_month in date_str:
            # Split the date string by spaces and extract parts
            parts = date_str.split(' ')
            if len(parts) == 2:  # Expected format: "Month Year"
                year = parts[1]
                # Ensure the year is four digits
                if len(year) == 2:
                    year = "20" + year
                return f"{num_month}.{year}"

    # Handle relative dates
    if 'pv' in date_str:
        days = int(date_str.split(' ')[0])
        new_date = specific_datetime - timedelta(days=days)
        return new_date.strftime('%m.%Y')
    elif 't' in date_str:
        hours = int(date_str.split(' ')[0])
        new_date = specific_datetime - timedelta(hours=hours)
        return new_date.strftime('%m.%Y')
    elif 'min' in date_str:
        minutes = int(date_str.split(' ')[0])
        new_date = specific_datetime - timedelta(minutes=minutes)
        return new_date.strftime('%m.%Y')

    # Debugging: Print date_str that couldn't be parsed
    print(f"Could not parse date: {date_str}")
    return None

# Function to convert Finnish timestamp to standard format (with four-digit year)
def convert_comment_timestamp(timestamp_str):
    # Split the timestamp string into components
    parts = timestamp_str.split(' ')
    
    # Extract the day, month name, year, and time
    day = parts[0].rstrip('.')
    month_name = parts[1]
    year = parts[2]
    time = parts[3]

    # Map the Finnish month name to a month number
    try:
        month_number = month_mapping[month_name.lower()]  # Direct mapping without default value
    except KeyError:
        raise ValueError(f"Invalid month name: {month_name}")
    
    # Construct the new timestamp in the format "YYYY-MM-DD HH:MM:SS"
    new_timestamp = f"{year}-{month_number.zfill(2)}-{day.zfill(2)} {time.replace('.', ':')}"

    return new_timestamp

# Apply the conversion functions to your DataFrame columns
data['Created At'] = data['Created At'].apply(convert_date)
data['Last Reply'] = data['Last Reply'].apply(convert_date)
data['Timestamp'] = data['Timestamp'].apply(convert_comment_timestamp)

# Apply the ISO format conversion to 'Created At' and 'Last Reply'
def convert_to_iso_format(date_str):
    parts = date_str.split('.')
    year = parts[1]
    month = parts[0]
    return f"{year}-{month}-01 00:00:00"

data['Created At'] = data['Created At'].apply(convert_to_iso_format)
data['Last Reply'] = data['Last Reply'].apply(convert_to_iso_format)


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21201 entries, 0 to 21200
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Created At              21201 non-null  object 
 1   Last Reply              21201 non-null  object 
 2   Visits                  21201 non-null  float64
 3   Replies                 21201 non-null  object 
 4   Users                   21201 non-null  object 
 5   Post Likes              21201 non-null  object 
 6   Links                   21201 non-null  float64
 7   Timestamp               21201 non-null  object 
 8   Likes                   21201 non-null  float64
 9   Comment                 21201 non-null  object 
 10  Comment ID              21201 non-null  object 
 11  Matched Companies Info  21201 non-null  object 
dtypes: float64(3), object(9)
memory usage: 1.9+ MB


In [15]:
# Set max_colwidth to None to prevent text truncation
pd.set_option('display.max_colwidth', None)

# Print the first 3 rows of the expanded DataFrame
print(data.head(3))

            Created At           Last Reply    Visits Replies Users  \
0  2020-12-01 00:00:00  2024-01-01 00:00:00   40283.0     229    53   
1  2020-06-01 00:00:00  2023-12-01 00:00:00  236849.0   1,4 k   230   
2  2020-06-01 00:00:00  2023-12-01 00:00:00  236849.0   1,4 k   230   

  Post Likes  Links         Timestamp  Likes  \
0      1,3 k   50.0  2020-12-23 21:29    4.0   
1      2,8 k   50.0   2020-06-11 7:46    5.0   
2      2,8 k   50.0  2020-06-11 11:22    5.0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21201 entries, 0 to 21200
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Created At              21201 non-null  object 
 1   Last Reply              21201 non-null  object 
 2   Visits                  21201 non-null  float64
 3   Replies                 21201 non-null  object 
 4   Users                   21201 non-null  object 
 5   Post Likes              21201 non-null  object 
 6   Links                   21201 non-null  float64
 7   Timestamp               21201 non-null  object 
 8   Likes                   21201 non-null  float64
 9   Comment                 21201 non-null  object 
 10  Comment ID              21201 non-null  object 
 11  Matched Companies Info  21201 non-null  object 
dtypes: float64(3), object(9)
memory usage: 1.9+ MB


In [17]:
df2 = data.copy()


In [18]:

# Assuming df2 is your DataFrame and it's already defined

# Updated function to convert mixed numeric representations to integers
def convert_to_int(value_str):
    # Check if the string contains "k" (indicating thousands)
    if 'k' in value_str.lower():
        # Remove the "k" and any commas, then convert to float and multiply by 1000
        return int(float(value_str.replace('k', '').replace(',', '')) * 1000)
    else:
        # For non-"k" values, first try to convert to float to handle any decimal points, then to int
        return int(float(value_str.replace(',', '')))

# Columns to apply the conversion
columns_to_convert = ['Visits', 'Replies', 'Users', 'Post Likes', 'Links', 'Likes']

# Applying the conversion function to the specified columns
for column in columns_to_convert:
    df2[column] = df2[column].astype(str).apply(convert_to_int)
# Converting time-related columns to datetime
df2['Created At'] = pd.to_datetime(df2['Created At'])
df2['Last Reply'] = pd.to_datetime(df2['Last Reply'])
df2['Timestamp'] = pd.to_datetime(df2['Timestamp'])

# Display the updated DataFrame to confirm the changes
print(df2.head())  # Adjust this to display the DataFrame as you prefer


  Created At Last Reply  Visits  Replies  Users  Post Likes  Links  \
0 2020-12-01 2024-01-01   40283      229     53       13000     50   
1 2020-06-01 2023-12-01  236849    14000    230       28000     50   
2 2020-06-01 2023-12-01  236849    14000    230       28000     50   
3 2020-06-01 2023-12-01  236849    14000    230       28000     50   
4 2019-06-01 2024-01-01   10685       26     12          55     12   

            Timestamp  Likes  \
0 2020-12-23 21:29:00      4   
1 2020-06-11 07:46:00      5   
2 2020-06-11 11:22:00      5   
3 2020-07-13 12:09:00      0   
4 2023-10-29 10:47:00      1   

                                                                                                                                                                                                                                                                                                                                                                                                  

In [19]:
df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21201 entries, 0 to 21200
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Created At              21201 non-null  datetime64[ns]
 1   Last Reply              21201 non-null  datetime64[ns]
 2   Visits                  21201 non-null  int64         
 3   Replies                 21201 non-null  int64         
 4   Users                   21201 non-null  int64         
 5   Post Likes              21201 non-null  int64         
 6   Links                   21201 non-null  int64         
 7   Timestamp               21201 non-null  datetime64[ns]
 8   Likes                   21201 non-null  int64         
 9   Comment                 21201 non-null  object        
 10  Comment ID              21201 non-null  object        
 11  Matched Companies Info  21201 non-null  object        
dtypes: datetime64[ns](3), int64(6), object(3)
memo

In [21]:
# Save DataFrame to a pickle file
df2.to_pickle('preprocessed_inderes_data.pkl')


In [22]:
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 9390.69834947586 seconds
